In [ ]:
import os
from ultralytics import YOLO
import cv2


# Paths and Configuration
video_name = 'VIDEO-2023-12-13-09-44-31'
file_extension = 'mp4'

VIDEOS_DIR = f'../raw_custom_data/{file_extension}'
OUTPUT_DIR = '../processed_videos_2'
os.makedirs(OUTPUT_DIR, exist_ok=True)

video_path = os.path.join(VIDEOS_DIR, f'{video_name}.{file_extension}')
video_path_out = os.path.join(OUTPUT_DIR, f'{video_name}_out.{file_extension}')

# Video Properties
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError(f"Error opening video file: {video_path}")

ret, frame = cap.read()
if not ret:
    raise ValueError("Failed to read the first frame. Check the video file.")

H, W, _ = frame.shape
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


# Output Codec Selection Based on Extension
codec_map = {
    'mp4': 'MP4V',  # Codec for .mp4
    'mov': 'MJPG'   # Codec for .mov
}

# Output Video Writer
fourcc = cv2.VideoWriter_fourcc(*codec_map.get(file_extension.lower(), 'MJPG'))  # Codec for output video
out = cv2.VideoWriter(video_path_out, fourcc, fps, (W, H))

# Load YOLO model
model_path = os.path.join('..', 'runs_new', 'detect', 'train', 'weights', 'best.pt')
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

# Load a model
model = YOLO(model_path)  # load a custom model

threshold = 0.3
frame_count = 0

class_colors = {
    0: (0, 255, 0),  # Green for class 0
    1: (0, 0, 255)   # Red for class 1
}

while ret:

    results = model(frame)[0]


    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            # Select color based on class_id
            color = class_colors.get(int(class_id), (255, 255, 255))  # Default to white if class_id is not found
            
            # Draw bounding box with the selected color
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            
            # Add label with class name and confidence
            label = f"{model.names[int(class_id)]} {score:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    out.write(frame)
    frame_count += 1
    print(f"Processed frame {frame_count}/{total_frames}")

    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved to {video_path_out}")